#### 切片
序列类型都支持切片操作
```python
seq[start:stop:step]
```
切取长度为(stop - start)# step = 1 <br>

step的正负表示切取方向，大小则表示切取间隔，在使用切片操作时，实际上是调用了`__getitem__(slice(start,stop,step))`方法，slice是切片对象 <br>

切片的其他特性方法：（在numpy等库中使用）
- 多维切片，使用以逗号分隔的多个索引或者切片
- 省略


In [1]:
l = list(range(10))
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
#就地修改序列
l[2:3] = [5] # 右侧必须是一个可迭代对象
l

[0, 1, 5, 3, 4, 5, 6, 7, 8, 9]

In [5]:
l[2:5] = [1,0] #数目无需匹配
l

[0, 1, 1, 0, 6, 7, 8, 9]

#### 对序列使用 + 和 *
使用 + 号，可以拼接两个序列，* 可以将原序列复制若干份，这两类操作，都不会改变原来的序列，而是构建一个全新的序列

In [8]:
a = [1,2,3]
b = ['a','b','c']
a+b,a,a*3,a

([1, 2, 3, 'a', 'b', 'c'], [1, 2, 3], [1, 2, 3, 1, 2, 3, 1, 2, 3], [1, 2, 3])

当使用*复制引用类对象时，需要注意，此时复制的引用与原来的引用指向的是同一个对象

In [9]:
import collections
Card = collections.namedtuple('Card','name color')
poker = Card('face','white')
c = [poker,1,2,[]]
d = c * 2
d,c

([Card(name='face', color='white'),
  1,
  2,
  [],
  Card(name='face', color='white'),
  1,
  2,
  []],
 [Card(name='face', color='white'), 1, 2, []])

In [12]:
d[3].append(2) # 可以看出，复制的内容和原内容指向同一块内存地址
d,c

([Card(name='face', color='white'),
  1,
  2,
  [2, 2],
  Card(name='face', color='white'),
  1,
  2,
  [2, 2]],
 [Card(name='face', color='white'), 1, 2, [2, 2]])

#### 序列的增量赋值
+=、*=的表现取决于它们的第一个操作对象，+=背后的特殊方法是`__iadd__`，如果操作对象没有实现该方法，会退而求其次选择`__add__`,*=背后的特殊方法则是`__imul__` <br>
当使用`__iadd__`时，增量赋值，会对操作对象进行就地修改，而`__add__`则会创建一个新的对象赋值给操作对象<br>
可变序列一般都实现了`__iadd__`等方法，如果是不可变序列，则因为其不可变特性，不支持就地修改
（str是个例，CPython对其进行了优化，在创建时会预留出额外的可拓展空间，因此进行增量操作时，不会涉及复制字符串到新位置的操作?）

In [14]:
a = [1,2,3]
c = [1,2,3]
b = ['a','b','c']
a+=b
c.extend(b)
a,c #+= 想当于 extend

([1, 2, 3, 'a', 'b', 'c'], [1, 2, 3, 'a', 'b', 'c'])

In [17]:
d = (1,2,3)
print(id(d))
d += tuple(b)
print(id(d)) # 创建了一个新对象返回，调用的是__add__

4346752688
4376556048


In [22]:
str1 = 'abcdef'
print(id(str1))
str1 +='g'
print(id(str1))
str1

4377041008
4401395248


'abcdefg'

#### list.sort 与 sorted
列表排序方法，`list.sort`会就地排序列表，返回值是None <br>
内置函数`sorted`则会新建一个列表作为返回值，可以接受任何形式的可迭代对象作为参数，包括不可变序列或生成器，最后返回都是一个列表<br>
```python
list.sort(cmp=None,key=None,reverse=False) #2.7版本
sorted(iterable,cmp=None,key=None,reverse=False) #2.7版本
#3.x版本移除了cmp参数，如果必须使用cmp，可以使用functools中的cmp_to_key，将cmp转为key传给函数
```
- cmp 比较使用的方法
- key 比较的元素
- reverse True=从大到小降序输出

In [23]:
a = [(1,'a'),(2,'c'),(4,'d'),(3,'b')]
a.sort() #默认以第一个元素，以字典序进行排序
a

[(1, 'a'), (2, 'c'), (3, 'b'), (4, 'd')]

In [40]:
a = [(1,'a'),(2,'c'),(4,'d'),(3,'b')]
b = sorted(a,key=lambda x:x[1])
a.sort(key=lambda x:x[1]) # 
a,b

([(1, 'a'), (3, 'b'), (2, 'c'), (4, 'd')],
 [(1, 'a'), (3, 'b'), (2, 'c'), (4, 'd')])

In [47]:
c = (4,5,2,1,3)
st1 = 'adcbe'
sorted(c),sorted(st1) #返回列表,原对象不可变

([1, 2, 3, 4, 5], ['a', 'b', 'c', 'd', 'e'])

In [61]:
from functools import cmp_to_key
a = [(1,'a'),(2,'c'),(4,'d'),(3,'b')]
b = sorted(a,key=cmp_to_key(lambda x,y:x[0]-y[0]))
b

[(1, 'a'), (2, 'c'), (3, 'b'), (4, 'd')]

#### 使用bisect管理已排序的序列
bisect模块有两个主要方法，一个是bisect，使用二分查找算法，返回插入的位置，该位置保证元素插入后，不影响原序列的排列 <br>
另一个是insort，同样使用二分查找算法找到插入位置，并将元素插入。
```python
bisect(literable,x,lo=0,hi=len(iterable)) # lo,hi可划定序列查找插入点的范围
insort(literable,x,lo=0,hi=len(iterable))
```

In [62]:
a = [4,5,6,2,1,3,7]
a.sort()
import bisect
bisect.bisect(a,3)#返回插入点的索引 

3

In [63]:
bisect.insort(a,3)
a

[1, 2, 3, 3, 4, 5, 6, 7]

In [67]:
print(bisect.bisect_left(a,3))#找到的位置在与该元素相等的元素左边 default
bisect.bisect_right(a,3) #找到的位置在与该元素相等的元素右边

2


4

In [68]:
bisect.insort_left(a,3)
bisect.insort_right(a,3)
a

[1, 2, 3, 3, 3, 3, 4, 5, 6, 7]